### Access Kimball Model

First we need to import our packages

In [1]:
import polars as pl
import deltalake

Then we save our storage options, which allow us to connect to the MinIO S3 buckets where our data are saved

In [2]:
storage_options = {
    "AWS_ACCESS_KEY_ID": "minio",
    "AWS_SECRET_ACCESS_KEY": "minio123",
    "AWS_REGION": "us-east-1",
    "AWS_ENDPOINT_URL": "http://10.5.0.5:9000",
    "AWS_ALLOW_HTTP": "TRUE",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "TRUE",
}

This code block allows us to see our currency dimension

In [3]:
# Set the location of the dim_currency table
dim_currency_uri = f"s3://gold/dim_currency"

# Read the contents of the dim_currency table into a DataFrame, passing in the storage options so we can connect to MinIO
dim_currency = pl.read_delta(
        dim_currency_uri,
        storage_options = storage_options
     )

# Print the DataFrame
print(dim_currency)

shape: (178, 3)
┌───────────────────────────────────┬───────────────┬───────────────────────────────────┐
│ dim_currency_id                   ┆ currency_code ┆ currency_name                     │
│ ---                               ┆ ---           ┆ ---                               │
│ str                               ┆ str           ┆ str                               │
╞═══════════════════════════════════╪═══════════════╪═══════════════════════════════════╡
│ cb3f8a65058da0b7c8524a4fb6c27f84… ┆ AED           ┆ UAE Dirham                        │
│ ef9094d7c60f06a7598d329d7cd00e6d… ┆ AFN           ┆ Afghani                           │
│ b5c7aed7cd2a308523e7d2847b781590… ┆ ALL           ┆ Lek                               │
│ 0fd3e4e71b62f7775696d50033366d82… ┆ AMD           ┆ Armenian Dram                     │
│ 428ac89ea662a034e7896f56358051dd… ┆ ANG           ┆ Netherlands Antillean Guilder     │
│ …                                 ┆ …             ┆ …                             

Similarily, we can connect to the fact table containing our rates

In [4]:
fct_rates_uri = f"s3://gold/fct_rates"

fct_rates = pl.read_delta(
        fct_rates_uri,
        storage_options = storage_options
     )

print(fct_rates)

shape: (7_608, 5)
┌────────────┬─────────────────────────────┬─────────────────────────────┬──────────────┬──────────┐
│ date       ┆ dim_currency_id_from        ┆ dim_currency_id_to          ┆ opening_rate ┆ movement │
│ ---        ┆ ---                         ┆ ---                         ┆ ---          ┆ ---      │
│ str        ┆ str                         ┆ str                         ┆ f32          ┆ f32      │
╞════════════╪═════════════════════════════╪═════════════════════════════╪══════════════╪══════════╡
│ 2014-02-16 ┆ 5e75885111cddfc6af2fde23355 ┆ a26cdf3a6e709124385d4d7eb9b ┆ 0.90596      ┆ 0.0      │
│            ┆ 681fa…                      ┆ ff6b8…                      ┆              ┆          │
│ 2014-02-17 ┆ 5e75885111cddfc6af2fde23355 ┆ a26cdf3a6e709124385d4d7eb9b ┆ 0.9039       ┆ 0.00005  │
│            ┆ 681fa…                      ┆ ff6b8…                      ┆              ┆          │
│ 2014-02-18 ┆ 5e75885111cddfc6af2fde23355 ┆ a26cdf3a6e709124385d4d7eb9b 

In [54]:
from datetime import date

dim_date = pl.date_range(date(2022, 1, 1), date.today(), "1d", eager=True).alias("date").to_frame()

In [55]:
print(dim_date)

shape: (778, 1)
┌────────────┐
│ date       │
│ ---        │
│ date       │
╞════════════╡
│ 2022-01-01 │
│ 2022-01-02 │
│ 2022-01-03 │
│ 2022-01-04 │
│ 2022-01-05 │
│ …          │
│ 2024-02-13 │
│ 2024-02-14 │
│ 2024-02-15 │
│ 2024-02-16 │
│ 2024-02-17 │
└────────────┘


In [48]:
dim_date = dim_date.with_columns([
    pl.col("date"),
    pl.col("date").dt.year().alias("year"),
    pl.col("date").dt.month().alias("month_number"),
    pl.col("date").dt.strftime("%B").alias("month_name"),
    pl.col("date").dt.day().alias("day_of_month"),
    pl.col("date").dt.strftime("%j").alias("day_of_year"),
    pl.col("date").dt.weekday().alias("weekday_number"),
    pl.col("date").dt.strftime("%A").alias("weekday_name"),
    pl.col("date").dt.strftime("%W").alias("week_number"),
]
)

In [50]:
print(dim_date)

shape: (60, 9)
┌────────────┬──────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ date       ┆ year ┆ month_numb ┆ month_name ┆ … ┆ day_of_yea ┆ weekday_n ┆ weekday_n ┆ week_numb │
│ ---        ┆ ---  ┆ er         ┆ ---        ┆   ┆ r          ┆ umber     ┆ ame       ┆ er        │
│ date       ┆ i32  ┆ ---        ┆ str        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│            ┆      ┆ i8         ┆            ┆   ┆ str        ┆ i8        ┆ str       ┆ str       │
╞════════════╪══════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 2022-01-01 ┆ 2022 ┆ 1          ┆ January    ┆ … ┆ 001        ┆ 6         ┆ Saturday  ┆ 00        │
│ 2022-01-02 ┆ 2022 ┆ 1          ┆ January    ┆ … ┆ 002        ┆ 7         ┆ Sunday    ┆ 00        │
│ 2022-01-03 ┆ 2022 ┆ 1          ┆ January    ┆ … ┆ 003        ┆ 1         ┆ Monday    ┆ 01        │
│ 2022-01-04 ┆ 2022 ┆ 1          ┆ January    ┆ … ┆ 004        ┆ 2         ┆

In [53]:
print(date.today())

2024-02-17
